In [ ]:
# Load in all the PGI and PGU values (mean+/-std) for each method and record it in a table
import pandas as pd

In [1]:
import os
os.chdir('..')

In [2]:
# print cwd
os.getcwd()

'/Users/nkroeger/Documents/Harvard/LLM_PostHocExplainer/LLM_Explainer'

In [3]:
import glob

In [80]:

def extract_method(filename):
    parts = filename.split('/')
    file_name = parts[-1]  # Get the actual filename without path
    name_parts = file_name.split('_')
    # Extract method and check for presence of 'preview' or versioning directly
    if 'preview' in file_name:
        method = name_parts[3] + '-0125-preview'
    else:
        method = name_parts[3]
    
    # Detect if 'v1' or 'pgicl' is in the filename and append
    if 'v1' in file_name:
        strategy = 'v1'
    elif 'pgicl' in file_name:
        strategy = 'pgicl'
    else:
        # If no known strategy is identified, return the method without strategy
        return method

    return f"{method}_{strategy}"

# Function to parse values from the simulated file content
def parse_values(content):
    lines = content.split('\n')
    pgi = lines[0].split(':')[1].strip()
    pgu = lines[1].split(':')[1].strip()
    return pgi, pgu

def pad_values(value):
    mean, std = value.split('+/-')
    mean = f"{float(mean):.3f}"
    std = f"{float(std):.3f}"
    return f"{mean}+/-{std}"

In [81]:
dataset_names = ['imdb', 'yelp', 'amazon_1000']
for dataset_name in dataset_names:
    faithfulness_dir = 'outputs/TextFaithfulnessResults/'+dataset_name+'/'
    # load all txt files in directory
    files = glob.glob(faithfulness_dir + '*.txt')
    # each .txt file is formatted as follows:
    # PGI:0.37+/-0.039
    # PGU:0.24+/-0.035
    
    file_contents = {}
    for file in files:
        with open(file, 'r') as f:
            file_contents[file] = f.read()
    
    # Create a DataFrame to store the data
    data = []
    for file in files:
        method = extract_method(file)
        pgi, pgu = parse_values(file_contents[file])
        data.append([method, pgi, pgu])
    
    # Apply padding to the PGI and PGU values
    data_padded = []
    for row in data:
        method, pgi, pgu = row
        pgi_padded = pad_values(pgi)
        pgu_padded = pad_values(pgu)
        data_padded.append([method, pgi_padded, pgu_padded])
    
    # Create DataFrame
    df = pd.DataFrame(data_padded, columns=['Method', 'PGI', 'PGU'])
    # sort by method name
    df = df.sort_values(by='Method')
    print(df)
    # Save the DataFrame to a CSV file
    df.to_csv('outputs/TextFaithfulnessResults/'+dataset_name+'/TextFaithfulnessResults_'+dataset_name+'.csv', index=False)


                                   Method            PGI            PGU
1   gpt-4-0125-preview-0125-preview_pgicl  0.329+/-0.045  0.214+/-0.039
4      gpt-4-0125-preview-0125-preview_v1  0.295+/-0.040  0.263+/-0.039
7                             gpt-4_pgicl  0.318+/-0.043  0.251+/-0.038
3                                gpt-4_v1  0.332+/-0.038  0.205+/-0.031
9                                      la  0.140+/-0.028  0.418+/-0.048
2                                      lc  0.178+/-0.033  0.339+/-0.042
5                                     ldl  0.208+/-0.034  0.367+/-0.046
0                                  lgshap  0.186+/-0.033  0.375+/-0.046
11                                   lgxa  0.229+/-0.037  0.373+/-0.045
12                                    lig  0.217+/-0.035  0.394+/-0.047
6                                lime1000  0.486+/-0.039  0.163+/-0.031
10                                 lime16  0.359+/-0.044  0.201+/-0.034
8                                  random  0.291+/-0.032  0.284+